There's two settings you need to change at the beginning of the semester to make this work.

1. Go to this [link](https://support.google.com/mail/answer/7126229?visit_id=638196126243117665-1776964642&hl=en&rd=1#zippy=%2Cstep-change-smtp-other-settings-in-your-email-client%2Cmy-email-client-is-crashing-or-emails-are-taking-too-long-to-download%2Csecurity-certificate-cn-error%2Ctoo-many-simultaneous-connections-error%2Ci-cant-sign-in-to-my-email-client): Go to "Step 2: Change SMTP & other settings in your email client > troubleshoot problems > allow less secure apps to access your account." Hit Yes.

2. Enable IMAP/POP access: Go to "Gmail settings > Forwarding and POP/IMAP" and make sure either POP or IMAP access is enabled.

Also, change the name and email in the next cell to yours.

To run it after each meeting, enter your date in the next cell, run it, enter your password, and run the rest. (You can go into 'Runtime' and hit 'Run all' or 'Run after')


In [ ]:
# Imports Python Packages
import smtplib
import getpass
import pandas as pd
from email.message import EmailMessage

# This was literally copied from the "code snippets (< >)"
# Look at the bottom left corner
# It asks for permission to access the google sheet (say yes)
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)


# THIS IS ALL YOU NEED TO DO:
# Sets Global Variables - Type the meeting name below and
# enter your password in the input prompt under the cell
meeting = 'Sept 12'
gmailUser = 'spandir@g.clemson.edu'
name = 'Sumanth Pandiri'
treasurerEmail = 'baylorn@g.clemson.edu'

gmailPassword = getpass.getpass("Enter Email Password: ")
server = smtplib.SMTP_SSL('smtp.gmail.com', 465)
server.login(gmailUser, gmailPassword)

In [ ]:
#This imports the sheet using that google spread stuff
worksheet = gc.open('Fall 2023 Attendance').sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()
print(rows)

# Convert to a DataFrame
df = pd.DataFrame.from_records(rows)

#show df
df

In [ ]:
#iterates over the first row and finds the column of the meeting specified
targetCol = 9999 #random number, obviously we wont have 9999 meetings
currentCol = 0
for element in df.iloc[1]:
    if (element == meeting):
        targetCol = currentCol
        print(targetCol)
    currentCol+=1

#throws an error if the meeting doesn't exist (probably a spelling issue)
if (targetCol == 9999):
    raise Exception("Meeting not found")


In [ ]:
#checks if it was a mandatory meeting, voting chapter, or chapter meeting and sets a boolean variable to T or F
mandatory = False
chapter = False
voting = False

if df[targetCol][0] == 'MC' or df[targetCol][0] == 'V' or df[targetCol][0] == 'C':
    if df[targetCol][0] == 'MC':
        mandatory = True
    if  df[targetCol][0] == 'V':
        voting = True
    chapter = True
    subject = 'Theta Tau Absence: ' + meeting + " Chapter"

else:
    if df[targetCol][0] == 'M':
        mandatory = True
    subject = 'Theta Tau Absence: ' + meeting

print(subject, mandatory, chapter, voting)

In [ ]:
#makes a list of all the rows of unexcused people
unexcusedList = []
index = 0
for item in df[targetCol]:
    if (item == 'u'):
        unexcusedList.append(index)
    index += 1
unexcusedList


In [ ]:
if voting:
    absenceType = "This absence was a voting chapter and will incur a $25 fine as detailed in Article III, Section 18 of the LG Bylaws in addition to any fee applicable under Article III Section 15. "
elif mandatory:
    andbsenceType = "This absence had mandatory attendance and will incur a $[?] fine as discussed. "
else :
    absenceType = ""

firstAbs = "This is your first unexcused absence and you have one remaining unexcused absence for this semester. "
secondAbs = "This is your second unexcused absence and you have no remaining unexcused absences for this semester. Any future unexcused absences will result in a fine of $10. "
thirdAbs = "You have now have 3 or more unexcused absences and will be fined $10 for this absence and any unexcused absences following."


In [ ]:
def bodyFunct(row):
    body = 'Dear ' + df.iloc[row][1] + ',\n\n' + absenceType

    unexcused = 0
    for element in df.iloc[row]:
        if element == 'u':
            unexcused += 1
    print(unexcused)

    match unexcused:
        case 1:
            absenceNum = firstAbs
        case 2:
            absenceNum = secondAbs
        case _:
            absenceNum = thirdAbs

    body += absenceNum
    body += "Please let me know within one week of this email being sent if you think this unexcused absence is an error. You will receive another email if you are marked unexcused again."
    body += '\n\n' + "Do not hesitate to reach out to either the Regent or myself if you have any questions or concerns."
    body += '\n\nIn H&T,\n' + name

    return body

In [ ]:
for item in unexcusedList:
    msg = EmailMessage()
    msg['Subject'] = subject
    msg['From'] = gmailUser
    msg['To'] =  df.iloc[item][2] + ', ' + treasurerEmail
    msg.set_content(bodyFunct(item))


    server = smtplib.SMTP_SSL('smtp.gmail.com', 465)

    server.login(gmailUser, gmailPassword)

    # Sending the message
    server.send_message(msg)

    server.quit()
